In [1]:
import sqlite3
import nest_asyncio
from aiogram import Bot, Dispatcher, executor, types
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton, ReplyKeyboardRemove
from aiogram.dispatcher import FSMContext
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.contrib.fsm_storage.memory import MemoryStorage
bot = Bot('6825504654:AAEmfx7f143tPkbgXqfCgbG68JY0fTvq5OU')
nest_asyncio.apply()

In [4]:
list_of_goals = []
list_of_comments = []
list_of_feedback = []

class Mystate(StatesGroup):
    new_goal = State()
    register = State()
    nextstep = State()
    feedback = State()
    onboarding = State()


# Menu of all buttons
menu = ReplyKeyboardMarkup(resize_keyboard=True)
menu.add(
    KeyboardButton('/Rules'),
    KeyboardButton('/Work_with_a_mentor'),
    KeyboardButton('/Set_career_goals')
)
menu.add(
    KeyboardButton('/Office_map'),
    KeyboardButton('/Teamwork'),
    KeyboardButton('/Development')
)
menu.add(
    KeyboardButton('/Additional')
)


storage = MemoryStorage()
dp = Dispatcher(bot, storage=storage)


# Connection to database and creating database
conn = sqlite3.connect('database.sql')
cur = conn.cursor()

cur.execute('CREATE TABLE IF NOT EXISTS employees (id INTEGER PRIMARY KEY, name TEXT, position TEXT, department TEXT, password TEXT)')
conn.commit()


# Button: Start
@dp.message_handler(commands=['start'])
async def start(message: types.Message):
    await message.answer('''Hello, to register please enter your full name, position, department and password in the format: 
    Name: <your name>
    Position: <your position>
    Department: <your department>
    Password: <your password>''')
    await Mystate.register.set()


# Info about user goes to database
@dp.message_handler(state=Mystate.register)
async def register_user(message: types.Message, state: FSMContext):
    data = message.text.split('\n')
    if len(data) >= 4:
        try:
            name = data[0].split(': ')[1]
            position = data[1].split(': ')[1]
            department = data[2].split(': ')[1]
            password = data[3].split(': ')[1]

            markup = ReplyKeyboardMarkup(resize_keyboard=True)
            markup.add(KeyboardButton('/Next_step'))

            cur.execute("INSERT INTO employees (name, position, department, password) VALUES (?, ?, ?, ?)", (name, position, department, password))
            conn.commit()

            await message.answer(f'You are registered as {name} in the position {position} in the department {department}.', reply_markup=markup)
            await Mystate.nextstep.set()
        except Exception as e:
            await message.answer(f'Error: {e}')
    else:
        await message.answer('Invalid input format. Please enter your full name, position, department, and password in the correct format.')


# Button: Next step
@dp.message_handler(state=Mystate.nextstep, commands=['Next_step'])
async def next_step(message: types.Message, state: FSMContext):
    data = message.text.split()
    await message.answer(
        '''You need to hold a meeting to resolve the issues of corporate mail, corporate laptop and badge.
        Here is the list of applications: 
        1) application for a parking space,
        2) application for access to the database,
        3) request for access to common tables,
        4) application to work from home
        Welcome to new company, select the section you are interested in''', 
        reply_markup=menu)
    await state.finish()



# Button: back
@dp.message_handler(commands=['Back'])
async def back(message: types.Message):
    await message.answer('Back!', reply_markup=menu)

    
    
# Button: Additional
@dp.message_handler(commands=['Additional'])
async def additional(message: types.Message):
    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add(KeyboardButton('/Onboarding_survey'),
               KeyboardButton('/Feedback_about_the_bot'))
    markup.add(KeyboardButton('/Back'))
    
    mes = 'Choose a section'
    await message.answer(mes, reply_markup=markup)
  
                              
                              
# Button: Onboarding_survey
@dp.message_handler(commands=['Onboarding_survey'])
async def onboarding_survey(message: types.Message):
    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add(KeyboardButton('/Back'))
    
    mes = '''Please, give feedback about your the adaptation process and the level of satisfaction'''
                              
    await message.answer(mes, reply_markup=markup)
    await Mystate.onboarding.set()
                              
#Function of onboaring
@dp.message_handler(state=Mystate.onboarding)
async def onboarding(message: types.Message, state: FSMContext):
    onboarding = message.text.strip()
    
    list_of_comments.append(onboarding)
                              
    await message.answer('''Your feedback has been added, thank you!''')
    await state.finish()
    
    
    
# Button: Feedback_about_the_bot
@dp.message_handler(commands=['Feedback_about_the_bot'])
async def feedback_about_the_bot(message: types.Message):
    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add(KeyboardButton('/Back'))
                              
    mes = '''Please, give feedback about our mentor-bot.'''
    await message.answer(mes, reply_markup=markup)
    await Mystate.feedback.set()
                              
# Function of feedback about bot 
@dp.message_handler(state=Mystate.feedback)
async def process_feedback(message: types.Message, state: FSMContext):
    feedback = message.text.strip()
                              
    list_of_feedback.append(feedback)
                              
    await message.answer('''Your feedback has been added, thank you!''')
    await state.finish()


               
# Button: Set career goals
@dp.message_handler(commands=['Set_career_goals'])
async def goals(message: types.Message):
    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add(KeyboardButton('/Back'),
               KeyboardButton('/New_goal'),
               KeyboardButton('/List_of_my_goals'))
    mes = 'Choose a section'
    await message.answer(mes, reply_markup=markup)4            

# Button: New goal
@dp.message_handler(commands=['New_goal'])
async def new_goal(message: types.Message):
    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add(KeyboardButton('/Back'))
    
    mes = 'Please, write your new goal. \nExample: become a middle data scientist - May 5'
    await message.answer(mes, reply_markup=markup)
    await Mystate.new_goal.set()

# Function of new goal
@dp.message_handler(state=Mystate.new_goal)
async def process_new_goal(message: types.Message, state: FSMContext):
    new_goal = message.text.strip()
    
    list_of_goals.append(new_goal)
    
    await message.answer(f'New goal "{new_goal}" has been added to the list.')
    await state.finish()

               
               
# Button: List of my goals
@dp.message_handler(commands=['List_of_my_goals'])
async def list_of_my_goals(message: types.Message):
    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add(KeyboardButton('/Back'))
    
    goals_text = '\n'.join(list_of_goals)
    
    mes = f'Your list:\n{goals_text}'
    await message.answer(mes, reply_markup=markup)
    
               
               
# Button: Clear goals
@dp.message_handler(commands=['Clear_goals'])
async def clear_goals(message: types.Message):
    global list_of_goals
    list_of_goals = []
    await message.answer('List is clear')
    


# Button: Rules
@dp.message_handler(commands=['Rules'])
async def rules(message: types.Message):
    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add(KeyboardButton('/Back'))
    await message.answer('First of all, you need to follow the safety rules at the workplace, namely:\ndo not eat at the workplace, do not leave bottles/cups that could spill and damage the workplace. Cybersecurity rules must be followed.\nYou also need to follow rules such as: do not shout, do not quarrel with colleagues and behave civilly', reply_markup=markup)



# Button: Work with a mentor
@dp.message_handler(commands=['Work_with_a_mentor'])
async def work_with_mentor(message: types.Message):
    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add(KeyboardButton('/Back'))
    mes = "Your mentor's name is Pavel Zaharov. He is assigned to you, you can approach him and ask questions. \nYou should also agree on regular meetings with him, where you will discuss your work schedule, development, and the mentor will tell you what meetings you need to attend."
    await message.answer(mes, reply_markup=markup)
    


# Button: Office map
@dp.message_handler(commands=['Office_map'])
async def office_map(message: types.Message):
    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add(KeyboardButton('/Back'))
    file = open('office_map.png', 'rb')
    await message.answer_photo(file)
    mes = 'It is your map of your office'
    await message.answer(mes, reply_markup=markup)

    

# Button: Teamwork
@dp.message_handler(commands=['Teamwork'])
async def teamwork(message: types.Message):
    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add(KeyboardButton('/Back'))
    mes = 'You need to meet the team and go to lunch with at least 3 new people, if you work remotely, then call them. \nThroughout the week, record the moments that are comfortable/uncomfortable at work, growth areas that would be interesting to develop'
    await message.answer(mes, reply_markup=markup)

    

# Button: Development
@dp.message_handler(commands=['Development'])
async def development(message: types.Message):
    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add(KeyboardButton('/Google'),
              KeyboardButton('/Courses'))
    markup.add(KeyboardButton('/Back'))
    mes = 'Useful links'
    await message.answer(mes, reply_markup=markup)



# Button: Google
@dp.message_handler(commands=['Google'])
async def google(message: types.Message):
    mes = 'https://www.google.com'
    await message.answer(mes, reply_markup=menu)
    
    
    
# Button: Courses
@dp.message_handler(commands=['Courses'])
async def courses(message: types.Message):
    mes = 'https://stepik.org/catalog'
    await message.answer(mes, reply_markup=menu)

        

executor.start_polling(dp) #обязательная для работы бота часть"


Goodbye!


In [5]:
cur.execute("SELECT * FROM employees")
rows = cur.fetchall()

for row in rows:
    print(row)

(1, 'Платон', 'джун', 'б2с', 'платон123')
(2, 'Platon ', 'jun DS', 'B2C', 'Platon123')
(3, 'Platon ', 'jun DS', 'B2C', 'Platon123')
(4, 'Platon ', 'jun DS', 'B2C', 'Platon123')
(5, 'Platon ', 'jun DS', 'B2C', 'Platon123')
(6, 'platon', 'jun', 'b2c', 'platon123')
(7, 'platon', 'jun', 'b2c', 'platon123')
(8, 'Platon', 'jun', 'b2c', 'platon123')
(9, 'Platon', 'jun', 'b2c', 'platon123')
(10, 'Platon', 'jun', 'b2c', 'platon123')
(11, 'Platon', 'jun', 'b2c', 'platon123')
(12, 'Platon', 'jun', 'b2c', 'platon123')
(13, 'Platon', 'jun', 'b2c', 'platon123')
(14, 'Platon', 'jun', 'b2c', 'platon123')
(15, 'Platon', 'jun', 'b2c', 'platon123')
(16, 'Platon', 'jun', 'b2c', 'platon123')
(17, 'Platon', 'jun', 'b2c', 'platon123')
(18, 'platon', 'jun', 'b2c', 'platon12345')
(19, 'platon', 'jun', 'b2c', 'platon12345')
(20, 'Bolotin Platon ', 'junior data scientist', 'B2C', 'Junior12345')
(21, 'Bolotin Platon ', 'junior data scientist', 'B2C', 'Junior123456789')
